In [18]:
import numpy as np
import theano
import theano.tensor as T
import lasagne

%matplotlib inline
import matplotlib.pyplot as plt

import skimage.transform
import sklearn.cross_validation
import pickle
import os

from lasagne.layers import DenseLayer, InputLayer, DropoutLayer, NonlinearityLayer, Pool2DLayer as PoolLayer
from lasagne.layers import Conv2DLayer as ConvLayer
# from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer # uncomment if you have cuDNN installed!
from lasagne.nonlinearities import softmax
from lasagne.updates import adam
from lasagne.utils import floatX
from nolearn.lasagne import NeuralNet
import pickle

In [4]:
np.random.seed(42)

In [5]:
CLASSES = ['pancakes', 'waffles']
LABELS = {cls: i for i, cls in enumerate(CLASSES)}

# Dataset

Images were downloaded from Google Image Search and placed in the directories './images/pancakes' and './images/waffles'

In [8]:
# Download and unpack set
!wget -N https://s3.amazonaws.com/emolson/pydata/images.tgz   
!tar -xf images.tgz

--2016-04-03 15:37:20--  https://s3.amazonaws.com/emolson/pydata/images.tgz
Resolving s3.amazonaws.com... 54.231.114.140
Connecting to s3.amazonaws.com|54.231.114.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12680660 (12M) [application/x-tar]
Saving to: 'images.tgz'

images.tgz          100%[===================>]  12.09M  2.92MB/s    in 5.8s    

2016-04-03 15:37:27 (2.08 MB/s) - 'images.tgz' saved [12680660/12680660]



In [11]:
from collections import OrderedDict
def define_layers():
    net = OrderedDict()
    net['input'] = InputLayer((None, 3, 224, 224), name='input')
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1, name='conv1_1')
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1, name='conv1_2')
    net['pool1'] = PoolLayer(net['conv1_2'], 2,)
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1)
    net['pool3'] = PoolLayer(net['conv3_3'], 2)
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1)
    net['pool4'] = PoolLayer(net['conv4_3'], 2)
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1)
    net['pool5'] = PoolLayer(net['conv5_3'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc7'] = DenseLayer(net['fc6'], num_units=4096)
    net['fc8'] = DenseLayer(net['fc7'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

In [12]:
layers = [v for k,v in define_layers().iteritems()]

In [14]:
net = NeuralNet(layers=layers, max_epochs=10, update=adam, update_learning_rate=0.0002, verbose=1)

In [17]:
# Download a pickle containing the pretrained weights
!wget -N https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg16.pkl

--2016-04-03 18:18:01--  https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg16.pkl
Resolving s3.amazonaws.com... 54.231.114.132
Connecting to s3.amazonaws.com|54.231.114.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553459517 (528M) [binary/octet-stream]
Saving to: 'vgg16.pkl'

vgg16.pkl           100%[===================>] 527.82M  5.10MB/s    in 83s     

2016-04-03 18:19:25 (6.37 MB/s) - 'vgg16.pkl' saved [553459517/553459517]



In [19]:
d = pickle.load(open('vgg16.pkl'))